In [2]:
from scipy.stats import invgamma
from scipy.stats import norm
from multiprocessing import Pool
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [3]:
def integral(estimate, ts):
    elements = np.ones(len(ts) - 1)
    for i in range(len(ts) - 1):
        elements[i] = (ts[i+1] - ts[i])*(estimate[i+1] + estimate[i])/2
    return sum(elements)

def sum1(beta):
    elements = np.ones(N)
    for i in range(N):
        elements[i] = p[i, 0] - beta * (p[i, 1] - ave_x)
    return np.sum(elements)

def sum2(alpha):
    elements = np.ones(N)
    for i in range(N):
        elements[i] = (p[i, 1] - ave_x)*(p[i, 0] - alpha)
    return np.sum(elements)

def sum3(alpha, beta):
    elements = np.ones(N)
    for i in range(N):
        elements[i] = (p[i, 0] - alpha - beta*(p[i, 1] - ave_x))**2
    return np.sum(elements)

def loglike(alpha, beta, sigma):
    return N*np.log(1/(sigma*np.sqrt(2*np.pi))) - sum3(alpha, beta) / (2*sigma)

def function1(w):
    if w < 100:
        np.random.seed(datetime.now().microsecond)
        
        ###### ここを変えてね！
        n = 10
        c = 2
        ######
        
        estimates = np.ones(n+1)
        ts = np.ones(n+1)

        for i in range(n+1):
            t = (i/n)**c
            ts[i] = t

            alphas = np.ones(sample_iter)
            betas = np.ones(sample_iter)
            sigmas = np.ones(sample_iter)
            
            if i == 0:
                alphas[0] = 3000
                betas[0] = 185
                sigmas[0] = 90000

            else:
                alphas[0] = np.mean(alpha_sample).item()
                betas[0] = np.mean(beta_sample).item()
                sigmas[0] = np.mean(sigma_sample).item()

            for j in range(sample_iter -1):

                location_alpha = (sigma_alpha*t*sum1(betas[-1]) + sigmas[-1]*mu_alpha) / (sigma_alpha * N*t + sigmas[-1])
                scale_alpha = np.sqrt((sigma_alpha * sigmas[-1]) / (sigma_alpha * N*t + sigmas[-1]))
                r = norm.rvs(loc = location_alpha, scale = scale_alpha)
                alphas[j+1] = r.item()

                location_beta = (sigma_beta * t * sum2(alphas[-1]) + sigmas[-1] * mu_beta) / (sigma_beta *t* var*N + sigmas[-1])
                scale_beta = np.sqrt((sigmas[-1] * sigma_beta) / (sigma_beta *t* var*N + sigmas[-1]))
                q = norm.rvs(loc = location_beta, scale = scale_beta)
                betas[j+1] = q.item()

                shape = N*t/2 + a
                invrate = 2*b / (b*t*sum3(alphas[-1], betas[-1]) + 2)
                s = invgamma.rvs(a = shape, scale = 1/ invrate)
                sigmas[j+1] = s.item()

            alpha_sample = alphas[burn_in:]
            beta_sample = betas[burn_in:len(betas)]
            sigma_sample = sigmas[burn_in:len(sigmas)]

            box = np.ones(len(alpha_sample))
            for k, l in enumerate(alpha_sample):
                box[k] = loglike(l, beta_sample[k], sigma_sample[k])

            estimates[i] = np.average(box)
        
        return estimates
    
def sum4(beta):
    elements = np.ones(N)
    for i in range(N):
        elements[i] = p[i, 0] - beta * (p[i, 2] - ave_z)
    return np.sum(elements)

def sum5(alpha):
    elements = np.ones(N)
    for i in range(N):
        elements[i] = (p[i, 2] - ave_z)*(p[i, 0] - alpha)
    return np.sum(elements)

def sum6(alpha, beta):
    elements = np.ones(N)
    for i in range(N):
        elements[i] = (p[i, 0] - alpha - beta*(p[i, 2] - ave_z))**2
    return np.sum(elements)    
    
def function2(w):
    if w < 100:
        np.random.seed(datetime.now().microsecond)
        
        ###### ここを変えてね！
        n = 10
        c = 2
        ######
        
        estimates = np.ones(n+1)
        ts = np.ones(n+1)

        for i in range(n+1):
            t = (i/n)**c
            ts[i] = t

            alphas = np.ones(sample_iter)
            betas = np.ones(sample_iter)
            sigmas = np.ones(sample_iter)
            
            if i == 0:
                alphas[0] = 3000
                betas[0] = 185
                sigmas[0] = 90000

            else:
                alphas[0] = np.mean(alpha_sample).item()
                betas[0] = np.mean(beta_sample).item()
                sigmas[0] = np.mean(sigma_sample).item()

            for j in range(sample_iter - 1):

                location_alpha = (sigma_alpha*t*sum4(betas[-1]) + sigmas[-1]*mu_alpha) / (sigma_alpha * N*t + sigmas[-1])
                scale_alpha = np.sqrt((sigma_alpha * sigmas[-1]) / (sigma_alpha * N*t + sigmas[-1]))
                r = norm.rvs(loc = location_alpha, scale = scale_alpha)
                alphas[j+1] = r.item()

                location_beta = (sigma_beta * t * sum5(alphas[-1]) + sigmas[-1] * mu_beta) / (sigma_beta *t* var2*N + sigmas[-1])
                scale_beta = np.sqrt((sigmas[-1] * sigma_beta) / (sigma_beta *t* var2*N + sigmas[-1]))
                q = norm.rvs(loc = location_beta, scale = scale_beta)
                betas[j+1] = q.item()

                shape = N*t/2 + a
                invrate = 2*b / (b*t*sum6(alphas[-1], betas[-1]) + 2)
                s = invgamma.rvs(a = shape, scale = 1/ invrate)
                sigmas[j+1] = s.item()

            alpha_sample = alphas[burn_in:]
            beta_sample = betas[burn_in:len(betas)]
            sigma_sample = sigmas[burn_in:len(sigmas)]

            box = np.ones(len(alpha_sample))
            for k, l in enumerate(alpha_sample):
                box[k] = loglike(l, beta_sample[k], sigma_sample[k])

            estimates[i] = np.average(box)
        
        return estimates

In [4]:
pine = pd.read_table("pine.txt", delim_whitespace = True)
p = pine.values
ave_x = np.average(p[:, 1]).item()
ave_z = np.average(p[:, 2]).item()

mu_alpha = 3000
sigma_alpha = 10**6
mu_beta = 185
sigma_beta = 10**4
a = 3
b = 1/(2*300**2)

N = np.shape(p)[0]
var = np.var(p[:, 1])
var2 = np.var(p[:, 2])

# ここをいじってね。
sample_iter = 1000
burn_in = 30
m = 10
core = 2


model1

In [5]:
if __name__ == '__main__':
    p = Pool(core) 
    result1 = p.map(function1, range(m))

In [6]:
result1

[array([-975.38366714, -612.42668923, -658.29542668, -679.21123331,
        -692.18224507, -698.92006902, -701.92413776, -705.79449061,
        -707.17305319, -709.49806737, -709.66603035]),
 array([-950.32152458, -613.14685931, -657.89760723, -680.39819475,
        -691.08558399, -698.14432316, -702.82692404, -704.94856373,
        -707.18330975, -708.66793318, -709.54340894]),
 array([-996.51160884, -613.29023244, -658.3891378 , -680.16780231,
        -691.24290752, -697.8639479 , -703.72764152, -705.54407379,
        -708.34601722, -708.39813048, -709.58104643]),
 array([-953.07480151, -612.6834424 , -657.36935828, -680.71424017,
        -691.06141503, -699.3710606 , -702.93899083, -705.46175403,
        -706.76873478, -708.31699589, -710.01287963]),
 array([-983.67608609, -612.5236283 , -657.98536184, -678.27524704,
        -691.36022397, -698.85384076, -702.56448045, -705.32017279,
        -707.60200437, -708.50417241, -709.05962451]),
 array([-938.1021554 , -614.65018267, -657.03

In [7]:
ts = [(i/10)**2 for i in range(11)]

expect1 = np.ones(10)
for i in range(10):
    expect1[i] = integral(result1[i], ts)

In [8]:
expect1

array([-700.79426843, -700.40383101, -700.9605367 , -700.52729571,
       -700.53928219, -700.60771555, -700.65756456, -700.48123203,
       -700.69806384, -700.83271647])

model2

In [9]:
if __name__ == '__main__':
    result2 = p.map(function2, range(m))

In [10]:
ts = [(i/10)**2 for i in range(11)]

expect2 = np.ones(10)
for i in range(10):
    expect2[i] = integral(result2[i], ts)

In [11]:
expect2

array([-700.72181525, -700.53943986, -700.85300765, -700.51260944,
       -700.73491173, -700.77328027, -700.77677517, -700.52847042,
       -700.88961816, -700.53515884])

BF21

In [12]:
bf_21 = []
for a,b in zip(expect1, expect2):
    bf_21.append(np.exp(b-a))

In [13]:
bf_21

[1.0751424617129306,
 0.87318410799839752,
 1.1135232118770031,
 1.0147946439821462,
 0.82231681815333513,
 0.84741501651074747,
 0.88762083434111649,
 0.95385998185749832,
 0.82567476851322552,
 1.34656598546509]